In [1]:
import sys
sys.path.append('..')
import numpy as np
import cupy as cp
import dask.array as da
import jax.numpy as jnp

# L2Norm

In [2]:
import numpy as np
from pycsou.func.penalty import L2Norm

for xp in [np, cp]:
    x = xp.arange(10)
    norm = L2Norm(dim=x.size)
    print(norm(x))
    # 16.881943016134134
    tau = 1.2; print(xp.allclose(norm.prox(x, tau=tau),xp.clip(1 - tau / norm(x), a_min=0, a_max=None) * x))
    # True
    lambda_ = 3; scaled_norm = lambda_ * norm; print(scaled_norm(x))
    # 50.645829048402405
    print(xp.allclose(scaled_norm.prox(x, tau=tau),xp.clip(1 - tau * lambda_ / norm(x), a_min=0, a_max=None) * x))
    # True

16.881943016134134
True
50.645829048402405
True
16.881943016134134
True
50.645829048402405
True


# SquaredL2Norm

In [3]:
from pycsou.func.penalty import SquaredL2Norm
from pycsou.linop.base import DenseLinearOperator

for xp in [np, cp]:
    x = xp.arange(10)
    norm = SquaredL2Norm(dim=x.size)
    print(norm(x))
    # 285.00000000000006
    print(xp.allclose(norm.gradient(x), 2 * x))
    # True
    lambda_=3; scaled_norm = lambda_ * norm
    print(scaled_norm(x))
    # 855.0000000000002
    print(xp.allclose(scaled_norm.gradient(x), 2 * lambda_ *  x))
    # True
    Gmat = xp.arange(100).reshape(10, 10)
    G = DenseLinearOperator(Gmat, is_symmetric=False)
    weighted_norm = norm * G
    print(xp.allclose(weighted_norm.gradient(x), 2 * Gmat.transpose() @ (Gmat @ x)))
    # True

285.00000000000006
True
855.0000000000002
True
True
285.00000000000006
True
855.0000000000002
True
True


# L2Ball

In [ ]:
## L2Ball
from pycsou.func.penalty import L2Ball
from pycsou.math.prox import proj_l2_ball

for xp in [np, cp]:
    x1 = xp.arange(10); x2 = x1 / xp.linalg.norm(x1)
    radius=10; ball = L2Ball(dim=x1.size, radius=radius)
    print(ball(x1), ball(x2))
    # (inf, 0)
    print(xp.allclose(ball.prox(x1,tau=1), proj_l2_ball(x1, radius=radius)), xp.linalg.norm(ball.prox(x1,tau=1)))
    # (True, 10.0)
    print(xp.allclose(ball.prox(x2,tau=1), x2))
    # True

inf 0
True 10.0
True
inf 0
True 9.999999999999998
True


# L1Norm

In [ ]:
from pycsou.func.penalty import L1Norm
from pycsou.math.prox import soft

for xp in [np, cp]:
    x = xp.arange(10)
    norm = L1Norm(dim=x.size)
    print(norm(x))
    # 45
    tau=1.2; print(xp.allclose(norm.prox(x, tau=tau),soft(x,tau=tau)))
    # True
    lambda_=3; scaled_norm = lambda_ * norm; print(scaled_norm(x))
    # 135
    print(xp.allclose(scaled_norm.prox(x, tau=tau),soft(x,tau=tau * lambda_)))
    # True

45
True
135
True
45
True
135
True


# SquaredL1Norm

In [ ]:
from pycsou.func.penalty import SquaredL1Norm

for xp in [np, cp]:
    x = xp.arange(10)
    norm = SquaredL1Norm(dim=x.size, prox_computation='sort')
    print(norm(x))
    # 2025
    norm2 = SquaredL1Norm(dim=x.size, prox_computation='root')
    print(xp.allclose(norm.prox(x, tau=1),norm2.prox(x, tau=1)))
    # True

2025
True
2025
True


# L1Ball

In [ ]:
from pycsou.func.penalty import L1Ball
from pycsou.math.prox import proj_l1_ball

for xp in [np, cp]:
    x1 = xp.arange(10); x2 = x1 / xp.linalg.norm(x1, ord=1)
    radius=10; ball = L1Ball(dim=x1.size, radius=radius)
    print(ball(x1), ball(x2))
    # (inf, 0)
    print(xp.allclose(ball.prox(x1,tau=1), proj_l1_ball(x1, radius=radius)), xp.linalg.norm(ball.prox(x1,tau=1), ord=1))
    # (True, 10.0)
    print(xp.allclose(ball.prox(x2,tau=1), x2))
    # True

inf 0
True 10.0
True
inf 0
True 10.0
True


# LInftyNorm

In [4]:
from pycsou.func.penalty import LInftyNorm

for xp in [np, cp]:
    x = xp.arange(10)
    norm = LInftyNorm(dim=x.size)
    print(norm(x))
    lambda_ = 3; scaled_norm = lambda_ * norm; print(scaled_norm(x))


9
27
9
27


# LInftyBall

In [5]:
from pycsou.func.penalty import LInftyBall
from pycsou.math.prox import proj_linfty_ball

for xp in [np, cp]:
    x1 = xp.arange(10); x2 = x1 / xp.linalg.norm(x1, ord=xp.inf)
    radius=8; ball = LInftyBall(dim=x1.size, radius=radius)
    print(ball(x1), ball(x2))
    # (inf, 0)
    print(xp.allclose(ball.prox(x1,tau=1), proj_linfty_ball(x1, radius=radius)), xp.linalg.norm(ball.prox(x1,tau=1), ord=xp.inf))
    # (True, 8.0)
    print(xp.allclose(ball.prox(x2,tau=1), x2))
    # True

inf 0
True 8.0
True
inf 0
True 8.0
True


# L21Norm

In [6]:
from pycsou.func.penalty import L21Norm, L2Norm, L1Norm

for xp in [np, cp]:
    x = xp.arange(10,dtype=xp.float32)
    groups = xp.concatenate((xp.ones(5),2*xp.ones(5)))
    group_norm = L21Norm(dim=x.size,groups=groups)
    print(type(group_norm))
    # <class 'pycsou.func.penalty.L21Norm'>
    print(group_norm(x))
    # 21.44594499772297
    l2_norm = L21Norm(dim=x.size,groups=xp.ones(x.size))
    print(type(l2_norm))
    # <class 'pycsou.func.penalty.L2Norm'>
    l1_norm = L21Norm(dim=x.size,groups=xp.arange(x.size)) # Also if groups = None
    print(type(l1_norm))
    # <class 'pycsou.func.penalty.L1Norm'>
    single_group_l2 = L2Norm(dim=x.size/2)
    tau = 0.5; print(xp.allclose(group_norm.prox(x,tau=tau),xp.concatenate((single_group_l2.prox(x[0:5], tau=tau),single_group_l2.prox(x[5:10],tau=tau)))))
    # True

<class 'pycsou.func.penalty.L21Norm'>
21.445946
<class 'pycsou.func.penalty.L2Norm'>
<class 'pycsou.func.penalty.L1Norm'>
True
<class 'pycsou.func.penalty.L21Norm'>
21.445946
<class 'pycsou.func.penalty.L2Norm'>
<class 'pycsou.func.penalty.L1Norm'>
True


# NonNegativeOrthant

In [9]:
from pycsou.func.penalty import NonNegativeOrthant
from pycsou.math.prox import proj_nonnegative_orthant

for xp in [np, xp]:
    x1 = xp.arange(10) - 5
    func = NonNegativeOrthant(dim=x1.size)
    print(func(x1), func(xp.abs(x1)))
    # (inf, 0)
    print(xp.allclose(func.prox(x1,tau=1), proj_nonnegative_orthant(x1)))
    # True
    print(xp.all(func.prox(x1,tau=1) >= 0))
    # True

inf 0
True
True
inf 0
True
True


# Segment

In [11]:
from pycsou.func.penalty import Segment
from pycsou.math.prox import proj_segment

for xp in [np, xp]:
    x1 = np.arange(10) - 3
    func = Segment(dim=x1.size, a=1, b=4)
    print(func(x1), func(np.clip(x1, a_min=1, a_max=4)))
    # (inf, 0)
    print(np.allclose(func.prox(x1,tau=1), proj_segment(x1, a=1, b=4)))
    # True
    print(func.prox(x1,tau=1))
    # array([1, 1, 1, 1, 1, 2, 3, 4, 4, 4])

inf 0
True
[1 1 1 1 1 2 3 4 4 4]
inf 0
True
[1 1 1 1 1 2 3 4 4 4]


# RealLine

In [14]:
from pycsou.func.penalty import RealLine

for xp in [np, cp]:
    x1 = xp.arange(10) + 1j
    func = RealLine(dim=x1.size)
    print(func(x1), func(xp.real(x1)))
    # (inf, 0)
    print(xp.allclose(func.prox(x1,tau=1), xp.real(x1)))
    # True

inf 0
True
inf 0
True


# ImagLine

In [15]:
from pycsou.func.penalty import ImagLine

for xp in [np, cp]:
    x1 = xp.arange(10) + 1j * xp.arange(10)
    func = ImagLine(dim=x1.size)
    print(func(x1), func(1j * xp.imag(x1)))
    # (inf, 0)
    print(xp.allclose(func.prox(x1,tau=1), xp.imag(x1)))
    # True

inf 0
True
inf 0
True


# LogBarrier

In [16]:
from pycsou.func.penalty import LogBarrier

for xp in [np, cp]:
    x1 = xp.arange(10)
    func = LogBarrier(dim=x1.size)
    print(func(x1), func(x1+2))
    # (inf, -17.502307845873887)
    print(xp.round(func.prox(x1,tau=1)))
    # array([1., 2., 2., 3., 4., 5., 6., 7., 8., 9.])

inf -17.502307845873887
[1. 2. 2. 3. 4. 5. 6. 7. 8. 9.]
inf -17.502307845873887
[1. 2. 2. 3. 4. 5. 6. 7. 8. 9.]


# ShannonEntropy

In [20]:
from pycsou.func.penalty import ShannonEntropy

for xp in [np, cp]:
    x1 = xp.arange(10); x2=xp.zeros(10); x2[0]=10
    func = ShannonEntropy(dim=x1.size)
    print(func(x1), func(x2))
    # (79.05697962199447, 23.02585092994046)
    print(xp.round(func.prox(x1,tau=2)))
    # array([0., 0., 1., 1., 1., 2., 2., 3., 3., 4.])

79.05697962199447 23.02585092994046
[0. 0. 1. 1. 1. 2. 2. 3. 3. 4.]
79.05697962199447 23.02585092994046


TypeError: operand type(s) all returned NotImplemented from __array_ufunc__(<ufunc '_lambertw'>, '__call__', array([ 0.18393972,  0.30326533,  0.5       ,  0.82436064,  1.35914091,
        2.24084454,  3.69452805,  6.09124698, 10.04276846, 16.55772598]), 0, 1e-08): 'ndarray', 'int', 'float'

In [ ]:
# lambertw is a scipy function not implemented in cupy

# QuadraticForm

In [21]:
from pycsou.linop import DenseLinearOperator
from pycsou.func.penalty import QuadraticForm

for xp in [np, cp]:
    rng = xp.random.default_rng(0)
    L =  rng.standard_normal(100).reshape(10,10)
    L = L.transpose() @ L #make definite positive
    Lop = DenseLinearOperator(L)
    F = QuadraticForm(dim=10,linop=Lop)
    x = xp.arange(10)
    print(xp.allclose(F(x), xp.dot(x, Lop @ x)))
    # True
    print(xp.allclose(F.gradient(x), 2 * Lop @ x))
    # True

True
True
True
True
